In [1]:
from pathlib import Path
import os
if '_fixed' not in locals():
    _fixed = os.chdir(os.path.dirname(os.getcwd()))
from ludwig import repo_root
from collections import Counter
from tqdm.notebook import tqdm
import random
from tabulate import tabulate
import json
import networkx as nx
import io
import matplotlib.pyplot as plt

In [2]:
from ludwig.util import vllm_Client

In [12]:
C = vllm_Client('8002')
C.prepare()
client = C.endpoint

In [13]:
C.model_name

'openai/gpt-oss-20b'

In [14]:
info = C._server_model_info()
model_id = info['data'][0]['id']
model_id

'/fast/fleeb/huggingface_cache/hub/models--openai--gpt-oss-20b/snapshots/2e8f8052ee2aeee907f76e08c08b9fdde8677ca8'

In [ ]:

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather in a given city",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"]
            },
        },
    }
]

chat = [{"role": "user", "content": "What's the weather in Berlin right now?"}]

response = client.chat.completions.create(
    model=model_id, #"openai/gpt-oss-20b",
    messages=chat,
    tools=tools
)
 
print(response.choices[0].message)

InternalServerError: Error code: 500 - {'error': {'message': 'Unexpected token 12606 while expecting start token 200006', 'type': 'Internal Server Error', 'param': None, 'code': 500}}

{'arguments': '{"location":"Berlin, Germany","unit":"celsius"}',
 'call_id': 'call_c1759da7b4c243ef81712d63d5d58c78',
 'name': 'get_current_weather',
 'type': 'function_call',
 'id': 'fc_c1759da7b4c243ef81712d63d5d58c78',
 'status': None}

In [18]:
response.model_dump()

{'id': 'resp_75fc123d7331406ea149cf1f37688229',
 'created_at': 1758188137.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': None,
 'model': '/fast/fleeb/huggingface_cache/hub/models--openai--gpt-oss-20b/snapshots/2e8f8052ee2aeee907f76e08c08b9fdde8677ca8',
 'object': 'response',
 'output': [{'id': 'rs_310ea9b6bf244129a232be2d0a82ad52',
   'summary': [],
   'type': 'reasoning',
   'status': None,
   'content': [{'text': 'User wants current weather in Berlin. We must use function get_current_weather. Provide location "Berlin, Germany"? Function expects "city, state". But city Berlin; no state. We can pass "Berlin, Germany". But function signature expects a location string. It\'s likely expecting "city, state" but we can pass city, country. There might be no strict enforcement. We\'ll ask function.',
     'type': 'reasoning_text'}],
   'encrypted_content': None},
  {'arguments': '{"location":"Berlin, Germany","unit":"celsius"}',
   'call_id': 'call_c1759da

In [1]:
# autoreload ludwig
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
if '_fixed' not in locals():
    _fixed = os.chdir(os.path.dirname(os.getcwd()))
from ludwig import repo_root
from collections import Counter
from tqdm.notebook import tqdm
import random
from tabulate import tabulate
import json
import networkx as nx
import io
import matplotlib.pyplot as plt

from typing import Optional
from ludwig.util.clients import Tool_Client, vllm_Client
from ludwig.util import ToolBase

In [2]:

class GetWeather(ToolBase):
    @property
    def name(self) -> str:
        return 'get_current_weather'

    def description(self) -> str:
        return "Get the current weather in a given location"

    def schema(self, style: str = None) -> 'JSONOBJ':
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description(),
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type":
                                "string",
                            "description":
                                "The city to find the weather for, e.g. 'San Francisco'"
                        },
                        "country": {
                            "type":
                                "string",
                            "description":
                                "the country (may be a 3 letter code) that the city is"
                                " in, e.g. 'AUT' which would mean 'Austria'"
                        },
                        "unit": {
                            "type": "string",
                            "description": "The unit to fetch the temperature in",
                            "enum": ["celsius", "fahrenheit"],
                            "default": "celsius"
                        }
                    },
                    "required": ["city", "country"]
                }
            }
        }

    def call(self, arguments: 'JSONOBJ', *, seed: Optional[int] = None) -> str:
        """
        Calls the tool with the given arguments and returns the result as a string.

        :param arguments: should adhere to the expected input specification. If not, will raise a `ToolError`.
        :param seed: optional to ensure deterministic behavior
        :raises: ToolError
        """
        assert isinstance(arguments, dict), f'Expected a dict, got {type(arguments)}'
        if 'city' not in arguments:
            raise ToolError("Missing 'city' in arguments")
        if 'country' not in arguments:
            raise ToolError("Missing 'country' in arguments")

        rng = random.Random(seed)

        temp = rng.randint(15, 35)

        city = arguments.get('city')
        fixed = {'Dallas': 31, 'Jakarta': 25, 'Barcelona': 28}
        if city in fixed:
            temp = fixed[city]

        country = arguments.get('country')

        unit = arguments.get('unit', 'celsius')
        if unit == 'fahrenheit':
            temp = temp * 9 / 5 + 32

        weather = rng.choice(['sunny', 'cloudy', 'rainy', 'dry'])
        # return f"The weather in {city}, {country} is {temp} degrees {unit} and {weather}."
        return json.dumps({'city': city, 'country': country, 'unit': unit, 'temp': temp, 'weather': weather})

class Client(Tool_Client, vllm_Client): pass

In [3]:

addr = '8002'
client = Client(addr=addr, tools=[GetWeather()])
client.prepare()

data = client.json()
print(data)

if not client.ping():
    raise RuntimeError("Client is not reachable.")
print()


{'base_url': 'http://localhost:8002/v1/', 'model_name': '/fast/fleeb/huggingface_cache/hub/models--openai--gpt-oss-20b/snapshots/2e8f8052ee2aeee907f76e08c08b9fdde8677ca8', 'max_tokens': None, 'temperature': None, 'top_p': None, 'seed': None, 'system_message': None, 'add_generation_prompt': True, 'continue_final_message': False, 'chat_template_path': None, 'addr': 'http://localhost:8002/v1/', 'tools': [{'type': 'function', 'function': {'name': 'get_current_weather', 'description': 'Get the current weather in a given location', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': "The city to find the weather for, e.g. 'San Francisco'"}, 'country': {'type': 'string', 'description': "the country (may be a 3 letter code) that the city is in, e.g. 'AUT' which would mean 'Austria'"}, 'unit': {'type': 'string', 'description': 'The unit to fetch the temperature in', 'enum': ['celsius', 'fahrenheit'], 'default': 'celsius'}}, 'required': ['city', 'country']}}

In [38]:

mxtokens = 512

chat = [{'role': 'user', 'content': "What is the weather in Dallas?"}]
r = client.step(chat, max_tokens=mxtokens)
# print(chat[-1]['content'])

In [39]:
chat

[{'role': 'user', 'content': 'What is the weather in Dallas?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'function': {'arguments': '{"city":"Dallas","country":"USA","unit":"celsius"}',
     'call_id': 'call_7bd9bba3197f4617b445a31ea9b54070',
     'name': 'get_current_weather',
     'type': 'function_call',
     'id': 'fc_7bd9bba3197f4617b445a31ea9b54070',
     'status': None},
    'arguments': '{"city":"Dallas","country":"USA","unit":"celsius"}',
    'call_id': 'call_7bd9bba3197f4617b445a31ea9b54070',
    'name': 'get_current_weather',
    'type': 'function_call',
    'id': 'fc_7bd9bba3197f4617b445a31ea9b54070',
    'status': None}]},
 {'role': 'tool',
  'content': '{"city": "Dallas", "country": "USA", "unit": "celsius", "temp": 31, "weather": "dry"}',
  'tool_call_id': 'fc_7bd9bba3197f4617b445a31ea9b54070',
  'name': 'get_current_weather'},
 {'role': 'assistant',
  'content': '**Dallas, USA – Current Weather**\n\n- **Temperature:** 31\u202f°C (88\u202f°F)  \n- **Con

In [8]:
# next(iter(client.tools.values())).schema()

In [15]:
model_id = client.endpoint.models.list().data[0].id
model_id

'/fast/fleeb/huggingface_cache/hub/models--openai--gpt-oss-20b/snapshots/2e8f8052ee2aeee907f76e08c08b9fdde8677ca8'

In [35]:
tools = [
    {
        "type": "function",
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location", "unit"],
        },
    },
]

tools = [{'type': 'function',
 'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': "The city to find the weather for, e.g. 'San Francisco'"},
    'country': {'type': 'string',
     'description': "the country (may be a 3 letter code) that the city is in, e.g. 'AUT' which would mean 'Austria'"},
    'unit': {'type': 'string',
     'description': 'The unit to fetch the temperature in',
     'enum': ['celsius', 'fahrenheit'],
     'default': 'celsius'}},
   'required': ['city', 'country']}}]


chat = [{"role": "user", "content": "What's the weather in Dallas right now?"}]

response = client.endpoint.responses.create(
    model=model_id,
    tools=tools,
    # input="What is the weather like in Boston today?",
    input=chat,
    tool_choice="auto",
)

# print('\n\n\n')
# print("Reasoning: ", response.output[0].content[0]['text'])

# print('\n\n\n')
# print("Function call: ",response.output[1].model_dump() )

r = response.model_dump()

chat += r['output']

for msg in chat:
    if msg.get('type') == 'reasoning':
        msg['id'] = 'rs_1'
        # del msg['status'], msg['summary']
        # del msg['status'], msg['id'], msg['summary']
        # del msg['id']#, msg['summary']

for item in response.output:
    if item.type == 'function_call':
        if item.name == "get_current_weather":
            arg = json.loads(item.arguments)

            result = GetWeather().call(arg)

            chat.append({
                'type': 'function_call_output',
                'call_id': item.call_id,
                'output': result,
            })
            

response2 = client.endpoint.responses.create(
    model=model_id,
    # instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=chat,
    tool_choice="auto",
)

response2.model_dump()

{'id': 'resp_92a74b6011254dbba2de785452224e18',
 'created_at': 1758191581.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': None,
 'model': '/fast/fleeb/huggingface_cache/hub/models--openai--gpt-oss-20b/snapshots/2e8f8052ee2aeee907f76e08c08b9fdde8677ca8',
 'object': 'response',
 'output': [{'id': 'rs_9d9c75e5796649f7b0aa9ab034f5d3a3',
   'summary': [],
   'type': 'reasoning',
   'status': None,
   'content': [{'text': 'The function returned temp 31 celsius dry. Should present to user.',
     'type': 'reasoning_text'}],
   'encrypted_content': None},
  {'id': 'msg_abb7241b7b7e41b7ab0f4f82124315a5',
   'content': [{'annotations': [],
     'text': '**Dallas – Current Weather (°C)**  \n- Temperature: **31\u202f°C**  \n- Condition: **Dry** 🔆  \n\nLet me know if you’d like more details (e.g., humidity, wind, forecast, or for a different unit).',
     'type': 'output_text',
     'logprobs': None}],
   'role': 'assistant',
   'status': 'completed',
   'type': 

In [21]:
print(response2.output[1].content[0].text)

**Current Weather in Dallas, USA**

- Temperature: **31 °C**  
- Conditions: **Rainy**  
- (If you prefer Fahrenheit, let me know!)


In [36]:
chat

[{'role': 'user', 'content': "What's the weather in Dallas right now?"},
 {'id': 'rs_1',
  'summary': [],
  'type': 'reasoning',
  'status': None,
  'content': [{'text': 'User asks: "What\'s the weather in Dallas right now?" We need to call get_current_weather function. Provide city, country, unit? Probably default celsius. Use city: "Dallas", country: "USA" (or maybe "US", "United States"? The function expects country: string, e.g., "AUT" (3 letter code). We\'ll use "USA". Return the function call.',
    'type': 'reasoning_text'}],
  'encrypted_content': None},
 {'arguments': '{"city":"Dallas","country":"USA"}',
  'call_id': 'call_46b8432fef9f4fa6ae56b999811f2882',
  'name': 'get_current_weather',
  'type': 'function_call',
  'id': 'fc_46b8432fef9f4fa6ae56b999811f2882',
  'status': None},
 {'type': 'function_call_output',
  'call_id': 'call_46b8432fef9f4fa6ae56b999811f2882',
  'output': '{"city": "Dallas", "country": "USA", "unit": "celsius", "temp": 31, "weather": "dry"}'}]

In [37]:
bool('')

False